In [1]:
import os
os.chdir('../')

In [73]:
from locale import atof, setlocale, LC_NUMERIC
setlocale(LC_NUMERIC, 'it_IT')

'it_IT'

In [2]:
import pandas as pd
from pathlib import Path
from PIL import Image
import json
import numpy as np
import subprocess

In [3]:
root_path = Path('./data/')

In [86]:
df = pd.read_csv(root_path / 'metadata' / 'final_matrix.csv')

# Take only the relevant part of the dataframe
df = df.iloc[4:,3:]
df = df.set_index('Unnamed: 3')
df.columns = df.iloc[0]
df = df.iloc[1:]
df.columns.name = ''
df.index.name = ''

# Remove NAs
df.dropna(how='all', axis=0, inplace=True)
df.dropna(how='all', axis=1, inplace=True)

# Convert all numbers to float
df = df.applymap(lambda x: atof(x))

In [94]:
style_df = pd.read_csv(root_path / 'metadata' / 'style.csv')
style_df = style_df[~style_df['File_name'].isna()]
style_df = style_df.sort_values('File_name').reset_index(drop=True)

content_df = pd.read_csv(root_path / 'metadata' / 'content.csv')
content_df = content_df[~content_df['File_name'].isna()]
content_df = content_df.sort_values('File_name').reset_index(drop=True)

In [95]:
n_random=200
content_indices = np.random.randint(0, content_df.shape[0], n_random)
style_indices = np.random.randint(0, style_df.shape[0], n_random)

In [96]:
content_path = root_path / 'content-images'
style_path = root_path / 'style-images'

metadata_path = root_path / 'output' / 'metadata'
metadata_path.mkdir(exist_ok=True, parents=True)

In [98]:
for i, (content_id, style_id) in enumerate(zip(content_indices, style_indices)):
    content = content_df.iloc[content_id]
    style = style_df.iloc[style_id]
    
    content_img_path = content_path / content['File_name']
    style_img_path = content_path / style['File_name']
    output_name = f'random_{i}'
    
    try:
        weight = df.loc[content['File_name'],style['File_name']]
    except:
        weight = 5e4
    
    metadata = {
                "description": "The most iconic pieces of art, reimagined by AI.",
                "image": "TBD",
                "name" : f"{content['Title']} X {style['Title']}",
                "animation_url": "TBD",
                "attributes": [
                    {
                        "trait_type": "Content",
                        "value": content['Title']
                    },
                    {
                        "trait_type": "Content Author",
                        "value": content['Author']
                    },
                    {
                        "trait_type": "Style",
                        "value": style['Title']
                    },
                    {
                        "trait_type": "Style Author",
                        "value": style['Author']
                    },
                    {
                        "trait_type": "Orientation",
                        "value": content['Orientation']
                    },
                    {
                        "trait_type": "File Name",
                        "value": output_name
                    },
                    {
                        "trait_type": "Style weight",
                        "value": weight
                    }
                ]
            }
    
    print(f"Processing content image: {content['File_name']}")
    print(f"Processing style: {style['File_name']}")
    print(f"Processing output name: {output_name}")
    print(json.dumps(metadata, indent=4))

#     !python neural_style_transfer.py --content_img_name {content['File_name']} --style_img_name {style['File_name']} --output_img_name {output_name} --output_path "random" --style_weight {weight}

#     file_metadata_path = metadata_path / (output_name + '.json')
#     with open(file_metadata_path, 'w') as f:
#         json.dump(metadata, f)
    
    print()

Processing content image: the_lady_of_shalott.jpg
Processing style: interieur_mit_zwei_maedchen_anagoria.jfif
Processing output name: random_0
{
    "description": "The most iconic pieces of art, reimagined by AI.",
    "image": "TBD",
    "name": "The Lady of Shalott X Interieur with Two Girls",
    "animation_url": "TBD",
    "attributes": [
        {
            "trait_type": "Content",
            "value": "The Lady of Shalott"
        },
        {
            "trait_type": "Content Author",
            "value": "John William Waterhouse"
        },
        {
            "trait_type": "Style",
            "value": "Interieur with Two Girls"
        },
        {
            "trait_type": "Style Author",
            "value": "Ludwig Kirchner"
        },
        {
            "trait_type": "Orientation",
            "value": "Landscape"
        },
        {
            "trait_type": "File Name",
            "value": "random_0"
        },
        {
            "trait_type": "Style weigh